In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./RAG_files/Test_demo03.csv")
data = loader.load()

In [2]:
import pandas as pd

ex = pd.read_csv('./RAG_files/Test_demo02.csv')
ex

/var/folders/xt/klwrj6ms3hj20z03dcgm8q6m0000gn/T/ipykernel_56175/2203280182.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Explorer Name,ID,Telephone,Email,Water Meter recorded number,Date,Time
0,Ray,1,021 0849 6271,kyrie19960324@gmail.com,286221,2/2/23,13:35
1,Dawn Keller,2,021 0609 7881,123889966@gmail.com,567890,3/2/23,12:05
2,Denis Hirano,3,023 4343 2323,ooll99776655@gmail.com,776542,4/2/23,10:58
3,Patrick,4,210878874,aasiyuafhn@gmail.com,109754,5/2/23,16:35
4,Lui Johnson,5,212863279,asdogah@gmail.com,800641,6/2/23,14:06
5,Colin HERBT,6,223452207,pllk@gmail.co.nz,665421,7/2/23,15:27
6,Colin HERBT,7,223452207,pllk@gmail.co.nz,280021,8/2/23,12:58


In [3]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
use_mlock = True  # Force system to keep model in RAM.

# Make sure the model path is correct for your system!

from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embed_model = HuggingFaceEmbeddings(
                model_name=embed_model_id,
                model_kwargs={'device': 'mps'}, 
                encode_kwargs={'device': 'mps'}
)

/Users/zwang/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


In [5]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

model_path="/Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9"


# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    #f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    use_mlock = True
)

user_question = "What does Ray submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"

from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
)

answer = qa_chain({"query": user_question})
print(answer)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model

 Based on the context provided, there is no information about what Ray submits looks like or whether he is opposing or supporting anything. Additionally, there is no information about Ray's telephone number in the context provided.{'query': 'What does Ray submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?', 'result': " Based on the context provided, there is no information about what Ray submits looks like or whether he is opposing or supporting anything. Additionally, there is no information about Ray's telephone number in the context provided."}



llama_print_timings:        load time =    5165.02 ms
llama_print_timings:      sample time =       4.98 ms /    45 runs   (    0.11 ms per token,  9032.52 tokens per second)
llama_print_timings: prompt eval time =    5164.73 ms /   126 tokens (   40.99 ms per token,    24.40 tokens per second)
llama_print_timings:        eval time =    4697.31 ms /    44 runs   (  106.76 ms per token,     9.37 tokens per second)
llama_print_timings:       total time =    9942.77 ms /   170 tokens


In [7]:
# user_question2 = "What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"
# answer = qa_chain({"query": user_question2})
# print(answer)

In [8]:
answer['result']

" Based on the context provided, there is no information about what Ray submits looks like or whether he is opposing or supporting anything. Additionally, there is no information about Ray's telephone number in the context provided."

In [9]:
user_question2 = "What does ID 2 submit looks like? List all the detail like: Is he Oppose or support? And what is he Telephone number?"
answer = qa_chain({"query": user_question2})
answer['result']

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Llama.generate: prefix-match hit


 Based on the provided context, there is no information about ID 2's submit look, whether they are oppose or support, and their telephone number. Therefore, it is impossible to answer the question with the given context.


llama_print_timings:        load time =    5165.02 ms
llama_print_timings:      sample time =       4.82 ms /    47 runs   (    0.10 ms per token,  9757.11 tokens per second)
llama_print_timings: prompt eval time =    3869.75 ms /    33 tokens (  117.27 ms per token,     8.53 tokens per second)
llama_print_timings:        eval time =    5052.90 ms /    46 runs   (  109.85 ms per token,     9.10 tokens per second)
llama_print_timings:       total time =    9010.03 ms /    79 tokens


" Based on the provided context, there is no information about ID 2's submit look, whether they are oppose or support, and their telephone number. Therefore, it is impossible to answer the question with the given context."